In [1]:
import numpy as np
from os import listdir, path
from PIL import Image
import time
import pickle
import tensorflow as tf
import numpy.random as rng
from sklearn.utils import shuffle
import os

In [3]:
t0 = time.time()
x_train, y_train = pickle.load(open('train_data.pkl', 'rb'))
print('data shapes:', x_train.shape, y_train.shape)
print('time elapsed:', time.time()-t0)

data shapes: (100000, 64, 64, 3) (100000,)
time elapsed: 1.7097628116607666


In [4]:
# try to use gpu
tf.test.is_gpu_available()

True

In [5]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras import backend as K
from keras.optimizers import Adam
from keras.regularizers import l2

left_input = Input((64, 64, 3))
right_input = Input((64, 64, 3))

conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(64, 64, 3))
for layer in conv_base.layers[:-1]: layer.trainable = False
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation="relu"))
print(model.summary())
    

encoded_l = model(left_input)
encoded_r = model(right_input)

L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid')(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

model = siamese_net

optimizer = Adam(lr = 0.0001)
model.compile(loss="binary_crossentropy",optimizer=optimizer)
print(model.summary())

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
Total params: 15,239,232
Trainable params: 524,544
Non-trainable params: 14,714,688
_________________________________________________________________
None
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
_______________________________________________